<a href="https://colab.research.google.com/github/fernandolazcar/OCR-aplicado-a-las-INES/blob/main/extractorOCR_para_INES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
# EXTRACTOR OCR INES

```


cargamos librerias para el uso del programa

In [ ]:
!pip install pdf2image
!pip install python-docx
!apt-get install -y poppler-utils
!pip install pytesseract
!sudo apt-get install -y tesseract-ocr
!apt-get install -y tesseract-ocr-spa
!pip install tablula-py
!pip install web3



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 3.5 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.4 [186 kB]
Fetched 186 kB in 0s (741 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 121918 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.4_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.4) ...
Setting up poppler-utils (22.02.0-2ubuntu0.4) ...
Processing triggers for man-db (2.10.2-1) ...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The follow

Sistema para la extraccion con tecnica OCR para INES

In [ ]:
import pytesseract
from PIL import Image
import os
import pandas as pd

# Ruta a la carpeta con las imágenes
image_folder_path = '/content/c'  # Cambia esto por la ruta correcta

# Verifica que la carpeta existe
if not os.path.exists(image_folder_path):
    print(f"La carpeta '{image_folder_path}' no existe. Por favor, verifica la ruta.")
else:
    # Obtiene una lista de todos los archivos de imagen en la carpeta
    image_files = [f for f in os.listdir(image_folder_path) if f.endswith('.jpg') or f.endswith('.jpeg') or f.endswith('.png')]
    image_files.sort()  # Asegura que los archivos estén en orden

    # Verifica que hay imágenes en la carpeta
    if not image_files:
        print(f"No se encontraron imágenes en la carpeta '{image_folder_path}'.")
    else:
        # Pregunta al usuario qué rango de imágenes desean procesar
        total_images = len(image_files)
        start_image = int(input(f"Ingresa el número de la primera imagen (1-{total_images}): ")) - 1
        end_image = int(input(f"Ingresa el número de la última imagen (1-{total_images}): ")) - 1

        if 0 <= start_image <= end_image < total_images:
            # Crear una lista para almacenar los resultados
            extracted_data = []

            # Procesa las imágenes seleccionadas y extrae el texto
            for image_number in range(start_image, end_image + 1):
                image_path = os.path.join(image_folder_path, image_files[image_number])
                try:
                    image = Image.open(image_path)
                    text = pytesseract.image_to_string(image, lang='spa')
                    extracted_data.append({'Image': image_files[image_number], 'ExtractedText': text})
                    print(f"Texto extraído de la imagen {image_number + 1}:\n{text}\n")
                except Exception as e:
                    print(f"Error al procesar la imagen '{image_files[image_number]}': {e}")

            # Convertir la lista de diccionarios a un DataFrame
            df = pd.DataFrame(extracted_data)

            # Mostrar el DataFrame
            print("DataFrame con texto extraído:")
            print(df)
        else:
            print("Rango de imágenes inválido.")


Ingresa el número de la primera imagen (1-2): 1
Ingresa el número de la última imagen (1-2): 2
Texto extraído de la imagen 1:
   

£

 

OEA

1DMEX1836577170<<0747116375842
8007057M2212315MEX<02<<12345<7
GOMEZ<VELAZQUEZ<<MARGARITA<<<<


Texto extraído de la imagen 2:
 

 

| 1IDMEX1745552595<<0856121167058
| 0003296M2812313MEX<00<<07491<1
RODRIGUEZ<MARQUEZ<<PAOLETTE<SJA


DataFrame con texto extraído:
                                       Image  \
0  Captura de pantalla 2024-05-28 001819.png   
1  Captura de pantalla 2024-05-28 164753.png   

                                       ExtractedText  
0     \n\n£\n\n \n\nOEA\n\n1DMEX1836577170<<07471...  
1   \n\n \n\n| 1IDMEX1745552595<<0856121167058\n|...  


busqueda de patron IDMEX

In [ ]:
import pandas as pd
import re


# Definir el patrón de texto a extraer
pattern = re.compile(r'DMEX\d+')

# Función para aplicar el patrón y extraer coincidencias
def extract_pattern(text):
    matches = pattern.findall(text)
    return matches[0] if matches else None

# Aplicar la función a la columna 'ExtractedText' y almacenar los resultados en una nueva columna
df['ExtractedPattern'] = df['ExtractedText'].apply(extract_pattern)

# Mostrar el DataFrame con la nueva columna
print("DataFrame con patrones extraídos:")
print(df)


DataFrame con patrones extraídos:
                                       Image  \
0  Captura de pantalla 2024-05-28 001819.png   
1  Captura de pantalla 2024-05-28 164753.png   

                                       ExtractedText ExtractedPattern  
0     \n\n£\n\n \n\nOEA\n\n1DMEX1836577170<<07471...   DMEX1836577170  
1   \n\n \n\n| 1IDMEX1745552595<<0856121167058\n|...   DMEX1745552595  


Limpieza de los datos que son duplicados

In [ ]:
import pandas as pd #libreria para la modificación de tablas


# Ver los datos originales
print("Datos originales:\n", df)

# Encontrar duplicados en una columna específica, por ejemplo, 'columna_a'
duplicados = df[df.duplicated('ExtractedPattern', keep=False)]

# Mostrar los datos duplicados
print("\nDatos duplicados en 'ExtractedPattern':\n", duplicados)

# Eliminar duplicados manteniendo la primera ocurrencia
df_sin_duplicados = df.drop_duplicates('ExtractedPattern', keep='first')

# Ver los datos sin duplicados
print("\nDatos sin duplicados:\n", df_sin_duplicados)


Datos originales:
           Image                                      ExtractedText  \
0  Reverso.jpeg   \n\n0309024H3112319MEX<00<<30657<8\nOLIVERA<R...   

  ExtractedPattern  
0             None  

Datos duplicados en 'ExtractedPattern':
 Empty DataFrame
Columns: [Image, ExtractedText, ExtractedPattern]
Index: []

Datos sin duplicados:
           Image                                      ExtractedText  \
0  Reverso.jpeg   \n\n0309024H3112319MEX<00<<30657<8\nOLIVERA<R...   

  ExtractedPattern  
0             None  


In [ ]:
df_sin_duplicados

,Image,ExtractedText,ExtractedPattern
0,Reverso.jpeg,\n\n0309024H3112319MEX<00<<30657<8\nOLIVERA<R...,None


Creación del Hash

In [ ]:
import pandas as pd
import hashlib



# Función para generar un hash
def generar_hash(dato):
    # Usando SHA-256
    return hashlib.sha256(dato.encode()).hexdigest()

# Aplicar la función de hash a la columna 'ExtractedPattern'
df_sin_duplicados['hashed_data'] = df_sin_duplicados['ExtractedPattern'].apply(generar_hash)

# Mostrar el DataFrame con la nueva columna de hashes
print(df_sin_duplicados)


                                       Image  \
0  Captura de pantalla 2024-05-28 001819.png   
1  Captura de pantalla 2024-05-28 164753.png   

                                       ExtractedText ExtractedPattern  \
0     \n\n£\n\n \n\nOEA\n\n1DMEX1836577170<<07471...   DMEX1836577170   
1   \n\n \n\n| 1IDMEX1745552595<<0856121167058\n|...   DMEX1745552595   

                                         hashed_data  
0  752af2c20ec418bcbc472aed35f6854840353ddb12e24e...  
1  85d4865025a289a2446e1aeabfd79f68dcfd3c6694f576...  


In [ ]:
df_sin_duplicados

,Image,ExtractedText,ExtractedPattern,hashed_data
0,Captura de pantalla 2024-05-28 001819.png,\n\n£\n\n \n\nOEA\n\n1DMEX1836577170<<07471...,DMEX1836577170,752af2c20ec418bcbc472aed35f6854840353ddb12e24e...
1,Captura de pantalla 2024-05-28 164753.png,\n\n \n\n| 1IDMEX1745552595<<0856121167058\n|...,DMEX1745552595,85d4865025a289a2446e1aeabfd79f68dcfd3c6694f576...
